In [6]:
import os
import numpy as np
import pandas as pd

PATH = '/home/seunghan9613/S-D-Mamba/results'
os.chdir(PATH)

In [7]:
settings = os.listdir()
print(len(settings))

1472


In [8]:
def get_result(DATASET):
    settings_dataset = [x for x in settings if DATASET in x]
    fnames = [os.path.join(x,'metrics.npy') for x in settings_dataset]
    fnames = [x for x in fnames if os.path.isfile(x)]
    
    result_dict = dict()
    for f in fnames:
        mae, mse, rmse, mape, mspe = np.load(f)
        result_dict[f] = mse
    
    df = pd.DataFrame(pd.Series(result_dict).sort_values())
 
    df.reset_index(inplace=True)
    df['data'] = df['index'].apply(lambda x:x.split('_')[0])
    df['L'] = df['index'].apply(lambda x:int(x.split('_')[2]))
    df['reg'] = df['index'].apply(lambda x:x.split('_')[5])
    df['lamb'] = df['index'].apply(lambda x:x.split('_0')[1].replace('metrics.npy','').replace('/',''))
    
    lambda_list = []
    k1_list = []
    mask_list = []
    conv_list = []
    wo_TD1_list = []
    wo_TD2_list = []

    for x in df['index']:
        if 'wo_conv' in x:
            conv_list.append(0)
        else:
            conv_list.append(1)
        if 'wo_TD1' in x:
            wo_TD1_list.append(0)
        else:
            wo_TD1_list.append(1)
        if 'wo_TD2' in x:
            wo_TD2_list.append(0)
        else:
            wo_TD2_list.append(1)
            
    for x in df['lamb']:
        if 'lambda' in x:
            lamb = x.split('lambda')[1].split('_')[0]
        else:
            lamb = 0
        
        if 'mask' in x:
            mask = x.split('mask')[1].split('_')[0]
        else:
            mask = 0
        if 'lambda' in x:
            lamb = x.split('lambda')[1].split('_')[0]
        else:
            lamb = 0
        
        if 'mask' in x:
            mask = x.split('mask')[1].split('_')[0]
        else:
            mask = 0
        lambda_list.append(float(lamb))
        mask_list.append(float(mask))
    df['lamb'] = lambda_list
    df['mask'] = mask_list
    df['conv'] = conv_list
    df['TD1'] = wo_TD1_list
    df['TD2'] = wo_TD2_list
    
    df.drop('index',axis=1,inplace=True)
    df.columns = ['mse','data','L','reg','lamb','mask','conv','TD1','TD2']
    df = df[['data','L','reg','lamb','mask','conv','TD1','TD2','mse']]
    df = df.pivot_table(index=['data', 'reg','lamb','mask','conv','TD1','TD2'], columns='L', values='mse').reset_index()
    #================================================================================#
    df = df[(df['reg']!='reg')&(df['reg']!='reg2')]
    #================================================================================#
    return df


# SOLAR

In [52]:
df = get_result(DATASET='solar')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
11,solar,reg4,0.0,0.0,0,1,1,0.207,0.24,0.261,0.265,0.243


In [17]:
solar = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
solar.loc['base'] = [0.205,0.237,0.258,0.260]
solar.loc['repr'] = [0.207,0.240,0.262,0.267]

solar['avg'] = solar.mean(axis=1)
solar.sort_values('avg').round(3)


L,96,192,336,720,avg
"(reg3, 0.0, 0, 1, 1)",0.206,0.239,0.259,NaN,0.235
base,0.205,0.237,0.258,0.260,0.240
"(reg3, 0.0, 0, 0, 1)",0.208,0.239,0.260,0.265,0.243
"(reg4, 0.0, 0, 1, 1)",0.207,0.240,0.261,0.265,0.243
"(reg3, 0.0, 1, 1, 1)",0.208,0.241,0.262,0.266,0.244
repr,0.207,0.240,0.262,0.267,0.244
"(reg4, 0.0, 0, 0, 1)",0.210,0.241,0.261,0.266,0.244
"(reg4, 0.0, 1, 1, 1)",0.210,0.243,0.263,0.266,0.246


# ETTh1:끝

In [54]:
df = get_result(DATASET='ETTh1')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
19,ETTh1,reg3,0.0,0.0,0,1,1,0.383,0.433,0.477,0.476,0.442


In [28]:
etth1 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
etth1.loc['base'] = [0.386,0.443,0.489,0.502]
etth1.loc['repr'] = [0.385,0.445,0.490,0.506]

etth1['avg'] = etth1.mean(axis=1)
etth1.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 1, 1, 1)",0.296,0.377,0.424,0.424,0.380
"(reg4, 0.0, 1, 1, 1)",0.297,0.378,0.425,0.424,0.381
"(reg4, 0.0, 0, 1, 1)",0.298,0.375,0.424,0.430,0.382
"(reg3, 0.0, 0, 1, 1)",0.299,0.375,0.423,0.431,0.382
"(reg4, 0.0, 0, 1, 0)",0.301,0.379,0.430,0.433,0.386
"(reg3, 0.0, 0, 1, 0)",0.302,0.379,0.429,0.434,0.386
"(reg3, 0.0, 0, 0, 1)",0.303,0.380,0.431,0.435,0.387
"(reg4, 0.0, 0, 0, 1)",0.302,0.382,0.432,0.434,0.387
base,0.386,0.443,0.489,0.502,0.455
repr,0.385,0.445,0.490,0.506,0.456


In [25]:
df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]

L                             96       192       336       720
reg  mask conv TD1 TD2                                        
reg3 0.0  0    0   1    0.384745  0.433494  0.476952  0.477429
               1   0    0.385290  0.434858  0.481337  0.482736
                   1    0.382514  0.432604  0.477481  0.476243
          1    1   1    0.384135  0.435505  0.483341  0.494436
reg4 0.0  0    0   1    0.385518  0.434038  0.478738  0.481348
               1   0    0.385869  0.435570  0.482763  0.486880
                   1    0.382809  0.432981  0.478761  0.482397
          1    1   1    0.383920  0.435681  0.483302  0.496324

# ETTh2:끝

In [57]:
df = get_result(DATASET='ETTh2')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
48,ETTh2,reg4,0.0,0.0,0,1,1,0.298,0.375,0.424,0.43,0.382


In [27]:
etth2 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
etth2.loc['base'] = [0.296,0.376,0.424,0.426]
etth2.loc['repr'] = [0.297,0.378,0.425,0.432]
etth2['avg'] = etth2.mean(axis=1)
etth2.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 1, 1, 1)",0.296,0.377,0.424,0.424,0.380
base,0.296,0.376,0.424,0.426,0.380
"(reg4, 0.0, 1, 1, 1)",0.297,0.378,0.425,0.424,0.381
"(reg4, 0.0, 0, 1, 1)",0.298,0.375,0.424,0.430,0.382
"(reg3, 0.0, 0, 1, 1)",0.299,0.375,0.423,0.431,0.382
repr,0.297,0.378,0.425,0.432,0.383
"(reg4, 0.0, 0, 1, 0)",0.301,0.379,0.430,0.433,0.386
"(reg3, 0.0, 0, 1, 0)",0.302,0.379,0.429,0.434,0.386
"(reg3, 0.0, 0, 0, 1)",0.303,0.380,0.431,0.435,0.387
"(reg4, 0.0, 0, 0, 1)",0.302,0.382,0.432,0.434,0.387


# ETTm1:끝

In [60]:
df = get_result(DATASET='ETTm1')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
23,ETTm1,reg3,0.0,0.0,0,1,1,0.336,0.376,0.412,0.475,0.4


In [18]:
ettm1 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
ettm1.loc['base'] = [0.333,0.376,0.408,0.475]
ettm1.loc['repr'] = [0.331,0.378,0.410,0.474]

ettm1['avg'] = ettm1.mean(axis=1)
ettm1.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg4, 0.0, 0, 0, 1)",0.329,0.376,NaN,NaN,0.353
"(reg4, 0.0, 1, 1, 1)",0.325,0.372,0.404,0.467,0.392
"(reg4, 0.0, 0, 1, 1)",0.326,0.374,0.409,0.473,0.395
"(reg3, 0.0, 0, 1, 1)",0.326,0.375,0.410,0.473,0.396
"(reg3, 0.0, 1, 1, 1)",0.329,0.375,0.409,0.472,0.396
"(reg3, 0.0, 0, 0, 1)",0.327,0.378,0.412,0.474,0.398
base,0.333,0.376,0.408,0.475,0.398
repr,0.331,0.378,0.410,0.474,0.398


# ETTm2:끝

In [63]:
df = get_result(DATASET='ETTm2')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
33,ETTm2,reg4,0.0,0.0,0,1,1,0.18,0.247,0.31,0.404,0.285


In [20]:
ettm2 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
ettm2.loc['base'] = [0.179,0.250,0.312,0.411]
ettm2.loc['repr'] = [0.182,0.252,0.313,0.413]

ettm2['avg'] = ettm2.mean(axis=1)
ettm2.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 0, 1, 1)",0.179,0.246,0.310,0.404,0.285
"(reg4, 0.0, 0, 1, 1)",0.179,0.247,0.310,0.404,0.285
"(reg3, 0.0, 1, 1, 1)",0.177,0.247,0.310,0.407,0.285
"(reg4, 0.0, 1, 1, 1)",0.178,0.248,0.311,0.407,0.286
base,0.179,0.250,0.312,0.411,0.288
repr,0.182,0.252,0.313,0.413,0.290


# Exchange:끝

In [66]:
df = get_result(DATASET='Exchange')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
14,Exchange,reg3,0.0,0.0,0,1,1,0.088,0.18,0.332,0.86,0.365


In [22]:
exchange = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
exchange.loc['base'] = [0.086,0.182,0.332,0.867]
exchange.loc['repr'] = [0.086,0.182,0.331,0.858]
exchange['avg'] = exchange.mean(axis=1)
exchange.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg4, 0.0, 0, 0, 1)",0.085,0.178,0.328,0.849,0.360
"(reg4, 0.0, 1, 1, 1)",0.085,0.178,0.330,0.854,0.361
"(reg3, 0.0, 0, 0, 1)",0.086,0.179,0.330,0.852,0.361
"(reg3, 0.0, 1, 1, 1)",0.084,0.178,0.330,0.855,0.362
"(reg4, 0.0, 0, 1, 1)",0.087,0.179,0.331,0.857,0.364
repr,0.086,0.182,0.331,0.858,0.364
"(reg3, 0.0, 0, 1, 1)",0.087,0.180,0.332,0.859,0.364
base,0.086,0.182,0.332,0.867,0.367


# Electricity:끝

In [68]:
df = get_result(DATASET='ECL')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg


In [24]:
ecl = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
ecl.loc['base'] = [0.139,0.159,0.176,0.204]
ecl.loc['repr'] = [0.139,0.165,0.177,0.214]
ecl['avg'] = ecl.mean(axis=1)
ecl.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 0, 1, 1)",0.139,0.158,0.177,0.201,0.169
base,0.139,0.159,0.176,0.204,0.170
"(reg4, 0.0, 0, 1, 1)",0.142,0.159,0.175,0.205,0.170
"(reg3, 0.0, 1, 1, 1)",0.140,0.160,0.181,0.204,0.171
"(reg4, 0.0, 1, 1, 1)",0.144,0.161,0.177,0.211,0.173
repr,0.139,0.165,0.177,0.214,0.174


# Traffic

In [72]:
df = get_result(DATASET='traffic')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
1,traffic,reg3,0.0,0.0,0,1,1,0.378,0.407,0.5,0.456,0.435


In [29]:
traffic = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
traffic.loc['base'] = [0.382,0.396,0.417,0.460]
traffic.loc['repr'] = [0.379,0.409,0.418,0.461]
traffic['avg'] = traffic.mean(axis=1)
traffic.sort_values('avg').round(3)

L,96,192,336,720,avg
base,0.382,0.396,0.417,0.460,0.414
repr,0.379,0.409,0.418,0.461,0.417
"(reg3, 0.0, 1, 1, 1)",0.382,0.413,0.415,0.462,0.418
"(reg4, 0.0, 1, 1, 1)",0.389,0.411,0.427,0.462,0.422


# Weather:끝

In [75]:
df = get_result(DATASET='weather')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg


In [32]:
weather = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
weather.loc['base'] = [0.165,0.214,0.274,0.350]
weather.loc['repr'] = [0.165,0.215,0.273,0.353]

weather['avg'] = weather.mean(axis=1)
weather.sort_values('avg').round(3)

L,96,192,336,720,avg
base,0.165,0.214,0.274,0.350,0.251
repr,0.165,0.215,0.273,0.353,0.252
"(reg3, 0.0, 1, 1, 1)",0.172,0.219,0.275,0.353,0.255
"(reg4, 0.0, 1, 1, 1)",0.173,0.221,0.276,0.355,0.256


# PEMS03

In [10]:
df = get_result(DATASET='PEMS03')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,12,24,48,96,avg
0,PEMS03,reg3,0.0000,0.0,1,1,1,0.066354,0.090002,0.168946,0.233092,0.139598
1,PEMS03,reg3,0.0001,0.0,1,1,1,0.065734,0.089528,0.169128,0.233509,0.139475
2,PEMS03,reg3,0.0010,0.0,1,1,1,0.066712,0.091037,0.171987,NaN,0.109912
3,PEMS03,reg4,0.0000,0.0,1,1,1,0.066529,0.089355,0.171963,0.548403,0.219062
4,PEMS03,reg4,0.0001,0.0,1,1,1,0.065991,0.089599,0.173156,0.228339,0.139271
5,PEMS03,reg4,0.0010,0.0,1,1,1,0.067116,0.091250,0.178545,0.215833,0.138186
6,PEMS03,reg4,0.0100,0.0,1,1,1,0.069094,NaN,NaN,NaN,0.069094


In [34]:
pems03 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems03.loc['base'] = [0.065,0.087,0.133,0.201]
pems03.loc['repr'] = [0.066,0.088,0.165,0.213]

pems03['avg'] = pems03.mean(axis=1)
pems03.sort_values('avg').round(3)

L,12,24,48,96,avg
base,0.065,0.087,0.133,0.201,0.122
repr,0.066,0.088,0.165,0.213,0.133
"(reg4, 0.0, 1, 1, 1)",0.066,0.089,0.172,0.216,0.136
"(reg3, 0.0, 1, 1, 1)",0.066,0.090,0.169,0.233,0.139


# PEMS04

In [13]:
df = get_result(DATASET='PEMS04')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)].round(3)


L,data,reg,lamb,mask,conv,TD1,TD2,12,24,48,96,avg
0,PEMS04,reg3,0.0,0.0,1,1,1,0.075,0.087,0.107,0.14,0.102


In [36]:
pems04 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems04.loc['base'] = [0.076,0.084,0.115,0.137]
pems04.loc['repr'] = [0.073,0.084,0.101,0.125]

pems04['avg'] = pems04.mean(axis=1) 
pems04.sort_values('avg').round(3)

KeyError: '[48, 96] not in index'

# PEMS07

In [17]:
df = get_result(DATASET='PEMS07')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,avg


In [16]:
pems07 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems07.loc['base'] = [0.063,0.081,0.093,0.117]
pems07.loc['repr'] = [0.060,0.082,0.100,0.117]

pems07['avg'] = pems07.mean(axis=1)
pems07.sort_values('avg').round(3)

KeyError: "None of [Index([12, 24, 48, 96], dtype='int64', name='L')] are in the [columns]"

# PEMS08

In [18]:
df = get_result(DATASET='PEMS08')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

In [19]:
pems08 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems08.loc['base'] = [0.076,0.104,0.167,0.245]
pems08.loc['repr'] = [0.076,0.109,0.173,0.271]

pems08['avg'] = pems08.mean(axis=1)
pems08.sort_values('avg').round(3)

L,12,24,48,96,avg
base,0.076,0.104,0.167,0.245,0.148
repr,0.076,0.109,0.173,0.271,0.157
"(reg3, 0.0, 1, 1, 1)",0.077,0.111,0.167,0.283,0.160
"(reg4, 0.0, 1, 1, 1)",0.079,0.116,0.178,0.273,0.162


In [96]:
etth1.index = etth1.index.astype('str')
etth2.index = etth2.index.astype('str')
ettm1.index = ettm1.index.astype('str')
ettm2.index = ettm2.index.astype('str')
exchange.index = exchange.index.astype('str')
ecl.index = ecl.index.astype('str')
weather.index = weather.index.astype('str')
traffic.index = traffic.index.astype('str')
pems03.index = pems03.index.astype('str')
#pems04.index = pems04.index.astype('str')
#pems07.index = pems07.index.astype('str')
pems08.index = pems08.index.astype('str')

In [100]:
etth1 = etth1.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
etth2 = etth2.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
ettm1 = ettm1.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
ettm2 = ettm2.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
exchange = exchange.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
ecl = ecl.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
weather = weather.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
traffic = traffic.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
pems03 = pems03.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
#pems04 = pems04.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
#pems07 = pems07.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
#pems08 = pems08.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]

In [101]:
etth1.index = ['add2','add1','paper','reproduced']
etth2.index = ['add2','add1','paper','reproduced']
ettm1.index = ['add2','add1','paper','reproduced']
ettm2.index = ['add2','add1','paper','reproduced']
exchange.index = ['add2','add1','paper','reproduced']
ecl.index = ['add2','add1','paper','reproduced']
weather.index = ['add2','add1','paper','reproduced']
traffic.index = ['add2','add1','paper','reproduced']
pems03.index = ['add2','add1','paper','reproduced']
#pems04.index = ['add2','add1','paper','reproduced']
#pems07.index = ['add2','add1','paper','reproduced']
#pems08.index = ['add2','add1','paper','reproduced']


In [108]:
etth1.index.name = 'etth1'
etth2.index.name = 'etth2'
ettm1.index.name = 'ettm1'
ettm2.index.name = 'ettm2'
exchange.index.name = 'exchange'
ecl.index.name = 'ecl'
weather.index.name = 'weather'
traffic.index.name = 'traffic'
pems03.index.name = 'pems03'
#pems04.index.name = 'pems04'
#pems07.index.name = 'pems07'
#pems08.index.name = 'pems08'

In [111]:
display(etth1.round(3))
display(etth2.round(3))
display(ettm1.round(3))
display(ettm2.round(3))
display(exchange.round(3))
display(ecl.round(3))
display(weather.round(3))
#display(traffic.round(3))
#display(pems03.round(3))

L,96,192,336,720,avg
etth1,,,,,
add2,0.384,0.436,0.483,0.494,0.449
add1,0.384,0.436,0.483,0.496,0.450
paper,0.386,0.443,0.489,0.502,0.455
reproduced,0.385,0.445,0.490,0.506,0.456


L,96,192,336,720,avg
etth2,,,,,
add2,0.296,0.377,0.424,0.424,0.380
add1,0.297,0.378,0.425,0.424,0.381
paper,0.296,0.376,0.424,0.426,0.380
reproduced,0.297,0.378,0.425,0.432,0.383


L,96,192,336,720,avg
ettm1,,,,,
add2,0.329,0.375,0.409,0.472,0.396
add1,0.325,0.372,0.404,0.467,0.392
paper,0.333,0.376,0.408,0.475,0.398
reproduced,0.331,0.378,0.410,0.474,0.398


L,96,192,336,720,avg
ettm2,,,,,
add2,0.177,0.247,0.310,0.407,0.285
add1,0.178,0.248,0.311,0.407,0.286
paper,0.179,0.250,0.312,0.411,0.288
reproduced,0.182,0.252,0.313,0.413,0.290


L,96,192,336,720,avg
exchange,,,,,
add2,0.084,0.178,0.330,0.855,0.362
add1,0.085,0.178,0.330,0.854,0.361
paper,0.086,0.182,0.332,0.867,0.367
reproduced,0.086,0.182,0.331,0.858,0.364


L,96,192,336,720,avg
ecl,,,,,
add2,0.140,0.160,0.181,0.204,0.171
add1,0.144,0.161,0.177,0.211,0.173
paper,0.139,0.159,0.176,0.204,0.170
reproduced,0.139,0.165,0.177,0.214,0.174


L,96,192,336,720,avg
weather,,,,,
add2,0.172,0.219,0.275,0.353,0.255
add1,0.173,0.221,0.276,0.355,0.256
paper,0.165,0.214,0.274,0.350,0.251
reproduced,0.165,0.215,0.273,0.353,0.252
